In [ ]:
# Copyright (c) 2024, ETH Zurich

In [ ]:
import sys
from pathlib import Path
import numpy as np
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import h5py

In [ ]:
rave_sim_dir = Path('path/to/rave-sim')
simulations_dir = Path('path/to/data/output')
scratch_dir = simulations_dir

In [ ]:
sys.path.insert(0, str(rave_sim_dir / "big-wave"))
import multisim
import config
import util
from nist_lookup.xraydb_plugin import xray_delta_beta

In [ ]:
# constants
h = 6.62607004 * 10**(-34) # planck constant in mˆ2 kg / s
c_0 = 299792458 # speed of light in m / s
eV_to_joule = 1.602176634*10**(-19)
N_A = 6.02214086 * 10**23 #[1/mol]

In [ ]:
def bin_me_res(data, x_axis, new_dx, axis=-2):
    """
    data must have dimensions:
    [x, z]
    """
    if axis!=-2:
        data = np.swapaxes(data, axis, -2)
    dx = abs(x_axis[1]-x_axis[0])
    binning = int(new_dx/dx)
    bin_range= int(data.shape[-2]/binning)
    if binning<2:
        print("no binnin necessary")
        return data
    sol = np.zeros((*data.shape[:-2],int(data.shape[-2]/binning),data.shape[-1]))
    new_x_grid = []
    for i in range(bin_range):
        sol[...,i,:] = np.sum(data[...,i*binning:(i+1)*binning,:], axis=-2)#[:,np.newaxis, :]
        new_x_grid.append(x_axis[...,i*binning+int(binning/2)])
    if axis!=-2:
        sol = np.swapaxes(sol, -2, axis)
    return sol, np.asarray(new_x_grid)

In [ ]:
import spekpy as spk
from scipy import interpolate

In [ ]:
s = spk.Spek(kvp=40, dk = 0.1, th = 90) # Create a spectrum
s.multi_filter((('Be', 0.15), ('Si', 0))) # Create a spectrum
k, f = s.get_spectrum(edges=True) # Get the spectrum

energyRange = [10000, 40000]
dE = 100
filtering = 0.000

energies = np.arange(10, 40+0.1, 0.1)*1e3


tube_spectrum_txt = interpolate.interp1d(k*1e3, f, fill_value = 'extrapolate')
spec_txt = tube_spectrum_txt(energies)

with h5py.File('path/to/simulation_spectrum_file.h5', 'w') as h5:
    h5.create_dataset('pdf', data =  spec_txt/ np.sum(spec_txt))
    h5.create_dataset('energy', data = energies)

In [ ]:
plt.plot(energies,spec_txt)

# tapering

In [ ]:
p = 1e-6
thick = 28.17e-6
s=1.0
d1=0.5
d2=0.00224

In [ ]:
config_dict = {
    "sim_params": {
        "N": 2**28,
        "dx": 0.23e-9,
        "z_detector": s,
        "detector_size": 3e-2,
        "detector_pixel_size_x": 1e-6,
        "detector_pixel_size_y": 1,
        "chunk_size": 256 * 1024 * 1024 // 16,  # use 256MB chunks
    },
    "dtype": "c8",
    "use_disk_vector": False,
    "save_final_u_vectors": False,
    "multisource": {
        "type": "points",
        "energy_range": [10000.0, 40000.0],
        "x_range": [-2.5e-6, 2.5e-6],
        "z": 0.0,
        "nr_source_points": 100,
        "seed": 1,
        "spectrum": 'path/to/simulation_spectrum_file.h5',
    },
    "elements": [
        {
            "type": "grating",
            "pitch": float(1e-6),
            "dc": [0.4, 0.5],
            "z_start": float(d1),#+0.1,
            "thickness": float(thick),
            "nr_steps": 20,
            "x_positions": [0.0],
            "substrate_thickness": (300 - float(thick)) * 1e-6,
            "mat_a": ["Si", 2.34],
            "mat_b": None,
            "mat_substrate": ["Si", 2.34],
        },
        {
            "type": "grating",
            "pitch": float(1e-6),
            "dc": [0.4, 0.5],
            "z_start": float(d1+d2),#+0.1,
            "thickness": float(thick),
            "nr_steps": 20,
            "x_positions": [float(i*1e-6) for i in np.linspace(0,1,11, endpoint=False)],
            "substrate_thickness": (300 - float(thick)) * 1e-6,
            "mat_a": ["Si", 2.34],
            "mat_b": None,
            "mat_substrate": ["Si", 2.34],
        },
    ],
}
sim_path = multisim.setup_simulation(config_dict, Path("."), simulations_dir)
for k in range(config_dict["multisource"]["nr_source_points"]):
    os.system(f"CUDA_VISIBLE_DEVICES=0 path/to/rave-sim/fast-wave/build-Release/fastwave -s {k} {sim_path}")

In [ ]:
computed = config.load(Path(sim_path / 'computed.yaml'))
wfs_tmp = util.load_wavefronts_filtered(sim_path, x_range=None, energy_range=None)
wf = []
for j in range(11):
    tmp=[]
    for k in range(len(computed["source_points"])):
        tmp.append(wfs_tmp[k][0][j])
    wf.append(tmp)
del wfs_tmp
wf = np.asarray(wf)

In [ ]:
wf.shape

In [ ]:
my_x = util.full_x_vector(len(wf[0,0]), 1e-6)

In [ ]:
def laplace(x, loc=0, scale=1):
    return np.exp(-abs(x-loc)/scale)/(2.*scale)/(np.exp(-abs(0)/scale)/(2.*scale))

In [ ]:
pixel_kernel = np.zeros_like(my_x)
limit=1
pixel_kernel[abs(my_x)<limit] = 1*laplace(my_x[abs(my_x)<limit], scale=3*16*1e-6)

In [ ]:
plt.plot(my_x/16.4e-6, pixel_kernel, "x")
# plt.xlim(-5e-5, 5e-5)
plt.xlim(-30, 30)

In [ ]:
wf_conv = np.asarray([np.convolve(wf[i].sum(axis=(0)), pixel_kernel[:], "same") for i in range(wf.shape[0])])

In [ ]:
wf224 = wf
wf_conv224 = wf_conv

In [ ]:
p = 1e-6
thick = 28.17e-6
s=1.0
d1=0.5
d2=0.00424

In [ ]:
config_dict = {
    "sim_params": {
        "N": 2**28,
        "dx": 0.23e-9,
        "z_detector": s,
        "detector_size": 3e-2,
        "detector_pixel_size_x": 1e-6,
        "detector_pixel_size_y": 1,
        "chunk_size": 256 * 1024 * 1024 // 16,  # use 256MB chunks
    },
    "dtype": "c8",
    "use_disk_vector": False,
    "save_final_u_vectors": False,
    "multisource": {
        "type": "points",
        "energy_range": [10000.0, 40000.0],
        "x_range": [-2.5e-6, 2.5e-6],
        "z": 0.0,
        "nr_source_points": 100,
        "seed": 1,
        "spectrum": 'path/to/simulation_spectrum_file.h5',
    },
    "elements": [
        {
            "type": "grating",
            "pitch": float(1e-6),
            "dc": [0.4, 0.5],
            "z_start": float(d1),#+0.1,
            "thickness": float(thick),
            "nr_steps": 20,
            "x_positions": [0.0],
            "substrate_thickness": (300 - float(thick)) * 1e-6,
            "mat_a": ["Si", 2.34],
            "mat_b": None,
            "mat_substrate": ["Si", 2.34],
        },
        {
            "type": "grating",
            "pitch": float(1e-6),
            "dc": [0.4, 0.5],
            "z_start": float(d1+d2),#+0.1,
            "thickness": float(thick),
            "nr_steps": 20,
            "x_positions": [float(i*1e-6) for i in np.linspace(0,1,11, endpoint=False)],
            "substrate_thickness": (300 - float(thick)) * 1e-6,
            "mat_a": ["Si", 2.34],
            "mat_b": None,
            "mat_substrate": ["Si", 2.34],
        },
    ],
}
sim_path = multisim.setup_simulation(config_dict, Path("."), simulations_dir)
for k in range(config_dict["multisource"]["nr_source_points"]):
    os.system(f"CUDA_VISIBLE_DEVICES=0 path/to/rave-sim/fast-wave/build-Release/fastwave -s {k} {sim_path}")

In [ ]:
computed = config.load(Path(sim_path / 'computed.yaml'))
wfs_tmp = util.load_wavefronts_filtered(sim_path, x_range=None, energy_range=None)
wf = []
for j in range(11):
    tmp=[]
    for k in range(len(computed["source_points"])):
        tmp.append(wfs_tmp[k][0][j])
    wf.append(tmp)
del wfs_tmp
wf = np.asarray(wf)

In [ ]:
wf.shape

In [ ]:
wf_conv = np.asarray([np.convolve(wf[i].sum(axis=(0)), pixel_kernel[:], "same") for i in range(wf.shape[0])])

In [ ]:
wf424 = wf
wf_conv424 = wf_conv

In [ ]:
p = 1e-6
thick = 28.17e-6
s=1.0
d1=0.5
d2=0.00624

In [ ]:
config_dict = {
    "sim_params": {
        "N": 2**28,
        "dx": 0.23e-9,
        "z_detector": s,
        "detector_size": 3e-2,
        "detector_pixel_size_x": 1e-6,
        "detector_pixel_size_y": 1,
        "chunk_size": 256 * 1024 * 1024 // 16,  # use 256MB chunks
    },
    "dtype": "c8",
    "use_disk_vector": False,
    "save_final_u_vectors": False,
    "multisource": {
        "type": "points",
        "energy_range": [10000.0, 40000.0],
        "x_range": [-2.5e-6, 2.5e-6],
        "z": 0.0,
        "nr_source_points": 100,
        "seed": 1,
        "spectrum": 'path/to/simulation_spectrum_file.h5',
    },
    "elements": [
        {
            "type": "grating",
            "pitch": float(1e-6),
            "dc": [0.4, 0.5],
            "z_start": float(d1),#+0.1,
            "thickness": float(thick),
            "nr_steps": 20,
            "x_positions": [0.0],
            "substrate_thickness": (300 - float(thick)) * 1e-6,
            "mat_a": ["Si", 2.34],
            "mat_b": None,
            "mat_substrate": ["Si", 2.34],
        },
        {
            "type": "grating",
            "pitch": float(1e-6),
            "dc": [0.4, 0.5],
            "z_start": float(d1+d2),#+0.1,
            "thickness": float(thick),
            "nr_steps": 20,
            "x_positions": [float(i*1e-6) for i in np.linspace(0,1,11, endpoint=False)],
            "substrate_thickness": (300 - float(thick)) * 1e-6,
            "mat_a": ["Si", 2.34],
            "mat_b": None,
            "mat_substrate": ["Si", 2.34],
        },
    ],
}
sim_path = multisim.setup_simulation(config_dict, Path("."), simulations_dir)
for k in range(config_dict["multisource"]["nr_source_points"]):
    os.system(f"CUDA_VISIBLE_DEVICES=0 path/to/rave-sim/fast-wave/build-Release/fastwave -s {k} {sim_path}")

In [ ]:
computed = config.load(Path(sim_path / 'computed.yaml'))
wfs_tmp = util.load_wavefronts_filtered(sim_path, x_range=None, energy_range=None)
wf = []
for j in range(11):
    tmp=[]
    for k in range(len(computed["source_points"])):
        tmp.append(wfs_tmp[k][0][j])
    wf.append(tmp)
del wfs_tmp
wf = np.asarray(wf)

In [ ]:
wf.shape

In [ ]:
wf_conv = np.asarray([np.convolve(wf[i].sum(axis=(0)), pixel_kernel[:], "same") for i in range(wf.shape[0])])

In [ ]:
wf624 = wf
wf_conv624 = wf_conv

In [ ]:
pf = lambda d1, d2, d3: (d1+d2+d3)*1e-6/(2*d2)

In [ ]:
pf(d1=21,d2=0.05, d3=10)

# Comparison

In [ ]:
import matplotlib.image as mpimg
# manual download of Fig. 1 from publication https://doi.org/10.1364/OE.477964
image_path = "getimagev2.jpg"
image = mpimg.imread(image_path)

In [ ]:
image.shape

In [ ]:
# Select the middle part of the image, around the yellow highlighted region in the provided image
# Assuming the middle part is around 200 pixels height centered
middle_section = image[195:205, :,:].sum(axis=-1)

# Average the intensity over the selected middle section (to reduce noise)
middle_section_avg = np.mean(middle_section, axis=0)[172:-47]

# Plot the intensity profile of the middle part
plt.figure(figsize=(10, 4))
plt.plot(middle_section_avg, color='blue')
plt.xlabel('Pixel index (x-axis)')
plt.ylabel('Intensity')
plt.title('Intensity profile of the middle part of the image')
plt.grid(True)
plt.show()

In [ ]:
interpol = interp1d(np.arange(len(middle_section_avg)), middle_section_avg)

In [ ]:
xs = np.linspace(0,len(middle_section_avg)-1, 2940)
dat = interpol(xs)

In [ ]:
# Plot the intensity profile of the middle part
plt.figure(figsize=(15, 4))
x_t = (xs-xs[-1]/2)*2902/2147*16.4/1000
plt.plot(x_t, dat, color='blue')
plt.xlabel('Pixel index (x-axis)')
plt.ylabel('Intensity')
plt.title('Intensity profile of the middle part of the image')
plt.grid(True)
plt.show()

In [ ]:
incident_angle = np.arctan(my_x/1)

# Plotting

In [ ]:
# matplotlib style
plt.style.use("default")

plt.rcParams["font.family"] = "serif"
plt.rcParams["font.size"] = 9
plt.rcParams["figure.dpi"] = 200
plt.rcParams["figure.constrained_layout.use"] = "True"


# Okabe-Ito palette
plt.rcParams["axes.prop_cycle"] = plt.cycler(
    color=[
        "#000000",
        "#E69F00",
        "#56B4E9",
        "#009E73",
        "#F0E442",
        "#0072B2",
        "#D55E00",
        "#CC79A7",
    ],
    marker=["", "", "", "", "D", "v", "v", "d"],
)

In [ ]:
visibility_color = "forestgreen"
fringe_color = "dimgray"
visibility_lw = 2.5
fringe_lw = 1
fringe_comp_lw = 1.5
fontsize = 10
fontsize_title = 14
visibility_bl = 0.04
visibility_ul = 0.22

In [ ]:
matcher = np.s_[:]
offset = 2

In [ ]:
c_fringe = "#000000"
c_visi = "#009E73"
c_visi_label = "#005740"
c_fcomp = "#CC79A7"
c_center = "blue"
c_side = "orangered"

In [ ]:
visis224 = (wf_conv224[:,:].max(axis=0)-wf_conv224[:,:].min(axis=0))/(wf_conv224[:,:].max(axis=0)+wf_conv224[:,:].min(axis=0))
fig, axes = plt.subplot_mosaic("AAAB;AAAC;DDDE;DDDF;GGGH;GGGI", figsize=(27/1.5/1.5,25/2/2))

# top plots
ax1 = axes["A"]
ax2 = axes["A"].twinx()
ax2.plot(my_x[50:-50]*1e3, np.convolve(visis224, pixel_kernel, "same")[50:-50]/np.convolve(visis224, pixel_kernel, "same")[50:-50].max()*visis224.max(), color=c_visi, lw=visibility_lw)
ax2.tick_params(axis='y', labelcolor=c_visi_label, labelsize = fontsize)
ax1.tick_params(axis='y', labelcolor="black", labelsize = fontsize)
ax1.tick_params(axis='x', labelcolor="black", labelsize = fontsize)

convo = wf_conv224[1,:]
convo = convo/convo.max()
ax1.plot(my_x[50:-50]*1e3, convo[50:-50], color=c_fringe, lw=fringe_lw)
ax1.plot(x_t[matcher]+offset*(x_t[1]-x_t[0]), (dat[matcher]+420)/(dat[matcher]+400).max(), linestyle=(0,(0.75,0.75)), color=c_fcomp, lw=fringe_comp_lw)
ax1.set_ylim(0.6,1.05)
ax1.set_ylabel("Intensity [a. u.]", fontsize = fontsize)
ax2.set_ylabel("Visibility", color=c_visi_label, fontsize = fontsize)
ax2.set_ylim(visibility_bl,visibility_ul)
ax1.set_title("Inter-grating distance 2.24 mm", fontsize = fontsize_title, fontweight='normal')
ax2.set_xlim(-14.7,14.7)

# top right
axes["B"].plot(incident_angle[abs(incident_angle)/(2*np.pi)*360<0.02]/np.pi/2*360, convo[abs(incident_angle)/(2*np.pi)*360<0.02], color=c_center)
ax1.plot(my_x[abs(incident_angle)/(2*np.pi)*360<0.02]*1e3, convo[abs(incident_angle)/(2*np.pi)*360<0.02], color=c_center)
axes["B"].plot(np.arctan((x_t[matcher]-2*(x_t[1]-x_t[0]))/1000)/(2*np.pi)*360, (dat[matcher]+400)/(dat[matcher]+400).max(), color=c_fcomp)
axes["B"].yaxis.set_visible(False)
axes["B"].spines['top'].set_visible(False)
axes["B"].spines['right'].set_visible(False)
axes["B"].spines['left'].set_visible(False)
axes["B"].tick_params(axis='x', which='both', bottom=True, top=False, labelbottom=True, labelsize = fontsize)
axes["B"].set_xlim(-0.02,0.02)
axes["B"].set_ylim(0.7,0.9)


# bottom right
axes["C"].plot(incident_angle[abs(incident_angle/(2*np.pi)*360-0.545)<0.025]/np.pi/2*360, convo[abs(incident_angle/(2*np.pi)*360-0.545)<0.025], color=c_side)
ax1.plot(my_x[abs(incident_angle/(2*np.pi)*360-0.545)<0.025]*1e3, convo[abs(incident_angle/(2*np.pi)*360-0.545)<0.025], color=c_side)
axes["C"].plot(np.arctan((x_t[matcher]+offset*(x_t[1]-x_t[0]))/1000)/(2*np.pi)*360, (dat[matcher]+420)/(dat[matcher]+400).max(), color=c_fcomp)
axes["C"].yaxis.set_visible(False)
axes["C"].spines['top'].set_visible(False)
axes["C"].spines['right'].set_visible(False)
axes["C"].spines['left'].set_visible(False)
axes["C"].tick_params(axis='x', which='both', bottom=True, top=False, labelbottom=True, labelsize = fontsize)
axes["C"].set_xlim(0.52,0.57)


# middle plots
visis424 = (wf_conv424[:,:].max(axis=0)-wf_conv424[:,:].min(axis=0))/(wf_conv424[:,:].max(axis=0)+wf_conv424[:,:].min(axis=0))
ax1 = axes["D"]
ax2 = axes["D"].twinx()
ax2.plot(my_x[50:-50]*1e3, np.convolve(visis424, pixel_kernel, "same")[50:-50]/np.convolve(visis424, pixel_kernel, "same")[50:-50].max()*visis424.max(), color=c_visi, lw=visibility_lw)
ax2.tick_params(axis='y', labelcolor=c_visi_label, labelsize = fontsize)
ax1.tick_params(axis='y', labelcolor="black", labelsize = fontsize)
ax1.tick_params(axis='x', labelcolor="black", labelsize = fontsize)

convo = wf_conv424[1,:]
convo = convo/convo.max()
ax1.plot(my_x[50:-50]*1e3, convo[50:-50], color=c_fringe, lw=fringe_lw)
ax1.set_ylim(0.6,1.05)
ax1.set_ylabel("Intensity [a. u.]", fontsize = fontsize)
ax2.set_ylabel("Visibility", color=c_visi_label, fontsize = fontsize)
ax2.set_ylim(visibility_bl,visibility_ul)
ax1.set_title("Inter-grating distance 4.24 mm", fontsize = fontsize_title, fontweight='normal')
ax2.set_xlim(-14.7,14.7)

# top right
axes["E"].plot(incident_angle[abs(incident_angle)/(2*np.pi)*360<0.02]/np.pi/2*360, wf_conv424[1,abs(incident_angle)/(2*np.pi)*360<0.02]*100, color=c_center)
ax1.plot(my_x[abs(incident_angle)/(2*np.pi)*360<0.02]*1e3, convo[abs(incident_angle)/(2*np.pi)*360<0.02], color=c_center)
axes["E"].yaxis.set_visible(False)
axes["E"].spines['top'].set_visible(False)
axes["E"].spines['right'].set_visible(False)
axes["E"].spines['left'].set_visible(False)
axes["E"].tick_params(axis='x', which='both', bottom=True, top=False, labelbottom=True, labelsize = fontsize)
axes["E"].set_xlim(-0.02,0.02)


# bottom right
axes["F"].plot(incident_angle[abs(incident_angle/(2*np.pi)*360-0.545)<0.025]/np.pi/2*360, wf_conv424[1,abs(incident_angle/(2*np.pi)*360-0.545)<0.025], color=c_side)
ax1.plot(my_x[abs(incident_angle/(2*np.pi)*360-0.545)<0.025]*1e3, convo[abs(incident_angle/(2*np.pi)*360-0.545)<0.025], color=c_side)
axes["F"].yaxis.set_visible(False)
axes["F"].spines['top'].set_visible(False)
axes["F"].spines['right'].set_visible(False)
axes["F"].spines['left'].set_visible(False)
axes["F"].tick_params(axis='x', which='both', bottom=True, top=False, labelbottom=True, labelsize = fontsize)
axes["F"].set_xlim(0.52,0.57)


# bottom plot
visis624 = (wf_conv624[:,:].max(axis=0)-wf_conv624[:,:].min(axis=0))/(wf_conv624[:,:].max(axis=0)+wf_conv624[:,:].min(axis=0))
ax1 = axes["G"]
ax2 = axes["G"].twinx()
ax2.plot(my_x[50:-50]*1e3, np.convolve(visis624, pixel_kernel, "same")[50:-50]/np.convolve(visis624, pixel_kernel, "same")[50:-50].max()*visis624.max(), color=c_visi, lw=visibility_lw)
ax2.tick_params(axis='y', labelcolor=c_visi_label, labelsize = fontsize)
ax1.tick_params(axis='y', labelcolor="black", labelsize = fontsize)
ax1.tick_params(axis='x', labelcolor="black", labelsize = fontsize)

convo = wf_conv624[1,:]
convo = convo/convo.max()
ax1.plot(my_x[50:-50]*1e3, convo[50:-50], color=c_fringe, lw=fringe_lw)
ax1.set_ylim(0.6,1.05)
ax1.set_ylabel("Intensity [a. u.]", fontsize = fontsize)
ax2.set_ylabel("Visibility", color=c_visi_label, fontsize = fontsize)
ax2.set_ylim(visibility_bl,visibility_ul)
ax1.set_title("Inter-grating distance 6.24 mm", fontsize = fontsize_title, fontweight='normal')
ax2.set_xlim(-14.7,14.7)
ax1.set_xlabel("distance from center / mm", fontsize = fontsize)

# top right
axes["H"].plot(incident_angle[abs(incident_angle)/(2*np.pi)*360<0.02]/np.pi/2*360, wf_conv624[1,abs(incident_angle)/(2*np.pi)*360<0.02], color=c_center)
ax1.plot(my_x[abs(incident_angle)/(2*np.pi)*360<0.02]*1e3, convo[abs(incident_angle)/(2*np.pi)*360<0.02], color=c_center)
axes["H"].yaxis.set_visible(False)
axes["H"].spines['top'].set_visible(False)
axes["H"].spines['right'].set_visible(False)
axes["H"].spines['left'].set_visible(False)
axes["H"].tick_params(axis='x', which='both', bottom=True, top=False, labelbottom=True, labelsize = fontsize)
axes["H"].set_xlim(-0.02,0.02)

# bottom right
axes["I"].plot(incident_angle[abs(incident_angle/(2*np.pi)*360-0.545)<0.025]/np.pi/2*360, wf_conv624[1,abs(incident_angle/(2*np.pi)*360-0.545)<0.025], color=c_side)
ax1.plot(my_x[abs(incident_angle/(2*np.pi)*360-0.545)<0.025]*1e3, convo[abs(incident_angle/(2*np.pi)*360-0.545)<0.025], color=c_side)
axes["I"].yaxis.set_visible(False)
axes["I"].spines['top'].set_visible(False)
axes["I"].spines['right'].set_visible(False)
axes["I"].spines['left'].set_visible(False)
axes["I"].tick_params(axis='x', which='both', bottom=True, top=False, labelbottom=True, labelsize = fontsize)
axes["I"].set_xlim(0.52,0.57)
axes["I"].set_xlabel("Incident angle / deg", fontsize = fontsize)

fig.tight_layout()
axes["A"].set_facecolor('whitesmoke')
axes["D"].set_facecolor('whitesmoke')
axes["G"].set_facecolor('whitesmoke')
